In [1]:
import pandas as pd
amazon = pd.read_csv(r"C:\Users\Administrator\Desktop\PADA_LAB\final_data\amazon_sentiment.csv", encoding='latin1')
audible = pd.read_csv(r"C:\Users\Administrator\Desktop\PADA_LAB\final_data\audible_sentiment.csv")
coursera = pd.read_csv(r"C:\Users\Administrator\Desktop\PADA_LAB\final_data\coursera_sentiment.csv")
hotel = pd.read_csv(r"C:\Users\Administrator\Desktop\PADA_LAB\final_data\hotel_sentiment.csv")

In [13]:
#인터렉션 데이터 분포 확인용용
import pandas as pd
from ydata_profiling import ProfileReport
scaled_audible = pd.read_csv(r"C:\Users\Administrator\Desktop\PADA_LAB\구조방정식\Count_SEM\scaled_data\amazon_scaled.csv")

# 프로파일링 리포트 생성 (수정된 파일 기반)
profile = ProfileReport(scaled_audible, title="Profiling Report")
profile.to_file("distribution_check_amazon.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## 전처리 코드(스케일링 포함)

In [12]:
import pandas as pd
import numpy as np
from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

def preprocess(df, scaling_name):
    removed_columns = df.select_dtypes(include=['object']).columns.tolist()
    
    # Helpfulness 열 따로 저장
    helpfullness_col = df['Helpfulness'] if 'Helpfulness' in df.columns else None
    removed_columns.append('Helpfulness')

    print("제외된 열:", removed_columns)
    
    df_clean = df.drop(columns=removed_columns)
    df_clean = df_clean.dropna()

    

    if scaling_name == 'standard':
            scaler = StandardScaler()
    elif scaling_name == 'minmax':
            scaler = MinMaxScaler()
    elif scaling_name == 'robust':
            scaler = RobustScaler()
        
    df_scaled = pd.DataFrame(scaler.fit_transform(df_clean), columns=df_clean.columns)
    
    # Helpfulness 열 다시 추가
    if helpfullness_col is not None:
        df_scaled['Helpfulness'] = helpfullness_col[df_clean.index].values
    
    return df_scaled

# 데이터셋 변환 (음수 처리 적용)
amazon_scaled = preprocess(amazon, 'standard')  # Box-Cox 변환
# audible_scaled = preprocess(audible, 'log')  # 로그 변환
# coursera_scaled = preprocess(coursera, 'minmax')  # MinMax 스케일링
# hotel_scaled = preprocess(hotel, 'robust')  # Robust 스케일링

# 결과 저장
output_path = "C:/Users/Administrator/Desktop/PADA_LAB/구조방정식/Count_SEM/scaled_data"
amazon_scaled.to_csv(f"{output_path}/amazon_scaled.csv", index=False)
# audible_scaled.to_csv(f"{output_path}\audible_scaled.csv", index=False)
# coursera_scaled.to_csv(f"{output_path}\coursera_scaled.csv", index=False)
# hotel_scaled.to_csv(f"{output_path}\hotel_scaled.csv", index=False)


제외된 열: ['Product_Name', 'Review_Title', 'Review_Text', 'Helpfulness']


## 포아송분포를 이용

In [12]:
print(amazon_scaled.columns)

Index(['Num_of_Ratings', 'Helpfulness', 'Is_Photo', 'Price', 'Time_Lapsed',
       'Average_Rating', 'Title_Length', 'Text_Length',
       'Deviation_Of_Star_Ratings', 'FOG_Index', 'Flesch_Reading_Ease',
       'Depth', 'Breadth', 'Valence', 'Arousal', 'Interaction'],
      dtype='object')
